<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Bayesian Linear Mixed Models (Genomic Data)</font> 
</div>

In [1]:
include("../../src/JWAS.jl")

JWAS

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [2]:
using JWAS,JWAS.Datasets,DataFrames,CSV

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [3]:
phenofile  = Datasets.dataset("example","phenotypes.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")
genofile   = Datasets.dataset("example","genotypes.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true)
pedigree   = get_pedigree(pedfile,separator=",",header=true);

coding pedigree...  10%|███                             |  ETA: 0:00:01

Finished!


coding pedigree... 100%|████████████████████████████████| Time: 0:00:00


In [4]:
head(phenotypes)

,ID,y1,y2,y3,x1,x2,x3,dam
1,a1,-0.06,3.58,-1.18,0.9,2,m,0
2,a2,-0.6,4.9,0.88,0.3,1,f,0
3,a3,-2.07,3.19,0.73,0.7,2,f,0
4,a4,-2.63,6.97,-0.83,0.6,1,m,a2
5,a5,2.31,3.5,-1.52,0.4,2,m,a2
6,a6,0.93,4.87,-0.01,5.0,2,f,a3


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Univariate Linear Mixed Model (Genomic data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [5]:
model_equation1  ="y1 = intercept + x1*x3 + x2 + x3 + ID" #+ dam";

In [6]:
R      = 1.0
model1 = build_model(model_equation1,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [7]:
set_covariate(model1,"x2");

<button type="button" class="btn btn-lg btn-primary">Step 5: Build Random or Fixed Effects</button> 

In [8]:
G1 = 1.0
G2 = 1.0#eye(2)
set_random(model1,"x2",G1);
set_random(model1,"x3",G1);
set_random(model1,"ID dam",pedigree,G2);

INFO: dam is not found in model equation 1.


<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [9]:
G3 =1.0
add_genotypes(model1,genofile,G3,separator=',',G_is_marker_variance=true);

5 markers on 10 individuals were added.


In [10]:
outputMCMCsamples(model1,"x3")

<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [11]:
out1=runMCMC(model1,phenotypes,methods="BayesC",Pi=0.95,estimatePi=true,chain_length=5000,output_samples_frequency=100);




MCMC Information:

methods                                      BayesC
chain_length                                   5000
burnin                                            0
starting_value                                false
printout_frequency                             5001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                                              0.95
estimatePi                                     true

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





INFO: The file MCMC_samples_residual_variance.txt is created to save MCMC samples for residual_variance.
INFO: The file MCMC_samples_polygenic_effects_variance.txt is created to save MCMC samples for polygenic_effects_variance.
INFO: The file MCMC_samples_y1_marker_effects.txt is created to save MCMC samples for y1_marker_effects.
INFO: The file MCMC_samples_marker_effects_variances.txt is created to save MCMC samples for marker_effects_variances.
INFO: The file MCMC_samples_pi.txt is created to save MCMC samples for pi.
INFO: The file MCMC_samples_x2_variances.txt is created to save MCMC samples for x2_variances.
INFO: The file MCMC_samples_x3_variances.txt is created to save MCMC samples for x3_variances.
running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:01


In [12]:
keys(out1)

Base.KeyIterator for a Dict{Any,Any} with 4 entries. Keys:
  "Posterior mean of marker effects"
  "Posterior mean of location parameters"
  "Posterior mean of Pi"
  "MCMC samples for: 1:x3"

In [13]:
model1.rndTrmVec[1].G

1×1 Array{Float64,2}:
 0.753825

In [14]:
out1["Posterior mean of location parameters"]

23×2 Array{Any,2}:
 "1:intercept : intercept"   6.15264  
 "1:x1*x3 : 0.9 * m"        -6.34333  
 "1:x1*x3 : 0.3 * f"        -7.06184  
 "1:x1*x3 : 0.7 * f"        -7.50179  
 "1:x1*x3 : 0.6 * m"        -7.57172  
 "1:x1*x3 : 0.4 * m"        -6.36046  
 "1:x1*x3 : 5.0 * f"        -3.46882  
 "1:x1*x3 : 0.5 * f"        -7.70675  
 "1:x1*x3 : 0.3 * m"        -9.42155  
 "1:x1*x3 : 0.2 * m"        -6.53968  
 "1:x2 : x2"                -0.120511 
 "1:x3 : m"                  0.141816 
 "1:x3 : f"                  0.0168737
 "1:ID : a1"                 0.0125568
 "1:ID : a3"                -0.198427 
 "1:ID : a6"                -0.270015 
 "1:ID : a7"                -0.10129  
 "1:ID : a2"                 0.273863 
 "1:ID : a4"                -0.143096 
 "1:ID : a8"                -0.217393 
 "1:ID : a5"                 0.595498 
 "1:ID : a10"                0.263868 
 "1:ID : a9"                -0.665439 

In [15]:
out1["MCMC samples for: 1:x3"]

51×2 Array{Any,2}:
   "1:x3 : m"    "1:x3 : f"
 -0.197881     -0.084814   
  0.332761     -0.0292767  
  0.498306     -1.63504    
  0.0793153    -0.349169   
  0.353        -0.979776   
  0.674899      0.420878   
  0.487354     -0.105524   
 -0.521934      0.0850107  
  1.21865       2.44988    
 -1.80449      -0.744066   
 -0.903768     -0.209353   
  1.70339      -0.0202008  
  ⋮                        
 -1.31588      -2.05326    
  1.00085      -0.0401776  
 -0.113128     -0.608831   
  0.612488     -0.289003   
 -0.196989      1.49691    
  0.746913      0.12156    
 -0.566589      1.08514    
 -0.262767      0.426406   
 -0.536189     -1.47933    
 -0.716376      0.0022575  
 -0.275866      0.751236   
  1.25281      -2.87613    

In [16]:
out2=runMCMC(model1,phenotypes,chain_length=5000,output_samples_frequency=100);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   5000
burnin                                            0
starting_value                                false
printout_frequency                             5001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





LoadError: [91mPlease build your model again using the function build_model().[39m

In [17]:
a=.01

0.01

In [18]:
a'

0.01

In [19]:
a=rand(10)

10-element Array{Float64,1}:
 0.747243
 0.532343
 0.455384
 0.250591
 0.284387
 0.34262 
 0.544634
 0.935642
 0.692632
 0.400449

In [20]:
a'

1×10 RowVector{Float64,Array{Float64,1}}:
 0.747243  0.532343  0.455384  0.250591  …  0.935642  0.692632  0.400449